In [1]:
import pandas as pd
import pickle
import my_func
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from sklearn.model_selection import KFold 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, accuracy_score, precision_score, recall_score, f1_score
from datetime import datetime, timedelta
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import seaborn as sns
import matplotlib.pyplot as plt
import time
from sklearn.metrics import confusion_matrix
import shap
from hyperopt import hp, fmin, tpe

sns.set()

fe_name_tr = 'churn_train_model_fe.pickle'
b_num_name_tr ='churn_train_model_b_num.pickle'
dpi_name_tr ='churn_train_model_dpi.pickle'

fe_name_te = 'churn_test_model_fe.pickle'
b_num_name_te ='churn_test_model_b_num.pickle'
dpi_name_te ='churn_test_model_dpi.pickle'

file_path = r'D:\BDL-Final\data-from-BDL' + '\\'

# settings for dataframe
pd.set_option('display.max_columns', None)

In [2]:
# Load dataframes
df_train_fe = pd.read_pickle(file_path+fe_name_tr)
df_train_dpi = pd.read_pickle(file_path+dpi_name_tr)
df_train_b_num = pd.read_pickle(file_path+b_num_name_tr)

df_test_fe = pd.read_pickle(file_path+fe_name_te)
df_test_dpi = pd.read_pickle(file_path+dpi_name_te)
df_test_b_num = pd.read_pickle(file_path+b_num_name_te)

In [38]:
df_train_fe

Ama_rchrgmnt_sum_max_mnt1  content_clc_mea_mnt1  content_cnt_max_mnt1  \
0                               0                   0.0             13.843750   
1                               0                   0.0             11.359375   
2                               0                   0.0             10.265625   
3                               0                   0.0              9.976562   
4                               0                   0.0              6.750000   
...                           ...                   ...                   ...   
149995                          0                   0.0              8.335938   
149996                          0                   0.0             14.039062   
149997                          0                   0.0             13.226562   
149998                          0                   0.0             19.531250   
149999                          0                   0.0             13.640625   

        voice_out_short_part_max_mnt1  voice_mts_in_nrest_part_std_mnt1  \
0                            1.480469                          0.000000   
1                            1.000000                          1.000000   
2                            0.000000                          0.000000   
3                            1.033203                          1.000000   
4                            1.000977                          1.000977   
...                               ...                               ...   
149995                       1.009766                          1.001953   
149996                       1.005859                          1.066406   
149997                       1.000977                          1.000977   
149998                       1.000977                          0.000000   
149999                       1.000977                          1.000000   

        num_act_days_max_mnt1  sms_roam_clc_min_mnt1  \
0                    5.324219                      0   
1                    5.324219                      0   
2                    5.324219                      0   
3                    5.324219                      0   
4                    5.324219                      0   
...                       ...                    ...   
149995               5.324219                      0   
149996               5.324219                      0   
149997               5.324219                      0   
149998               5.324219                      0   
149999               5.324219                      0   

        voice_in_cmpttrs_avg_durmin_mnt1  com_num_part_mea_mnt1  \
0                              19.046875               1.657227   
1                               0.000000               1.000977   
2                               0.000000               1.015625   
3                               0.000000               1.002930   
4                              28.859375               1.047852   
...                                  ...                    ...   
149995                          0.000000               1.334961   
149996                          0.000000               1.208008   
149997                          0.000000               1.037109   
149998                         15.460938               1.276367   
149999                         11.492188               1.112305   

        pay_avg_mea_mnt1  voice_out_tar_dur_std_mnt1  \
0               0.000000                   23.140625   
1              15.015625                   87.625000   
2              11.617188                   54.843750   
3              11.617188                   42.843750   
4               0.000000                   70.187500   
...                  ...                         ...   
149995         14.328125                   66.437500   
149996         12.789062                   72.375000   
149997          2.207031                   86.937500   
149998          9.109375                   54.593750   
149999         13.843750                

In [ ]:

import lightgbm as lgb
import pickle
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt


# Define the file name of the pickle file
pickle_file = "LGBMClassifier_24062023_15_30.pickle"
#pickle_file = "LGBMClassifier_28062023_20_01.pickle"

# Load the model
model, features = my_func.load_pickle(pickle_file)

# Prepare the test data (X_test)


y_test = df_test_fe['target']
X_test = df_test_fe.drop(['target', 'abon_id'], axis=1)
#X_test.fillna(-1, inplace=True)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate the probabilities of the positive class
y_pred_proba = model.predict_proba(X_test)[:, 1]

# Calculate the ROC-AUC score
auc = roc_auc_score(y_test, y_pred_proba)
print(f"Test AUC = {auc:.3f}")

# Plot the ROC-AUC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
plt.plot(fpr, tpr, color='blue', label="ROC Curve (AUC = {:.3f})".format(auc))
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC-AUC Curve")
plt.legend()
plt.show()


In [3]:
# Define the file name of the pickle file
pickle_file = "LGBMClassifier_24062023_15_30.pickle"
#pickle_file = "LGBMClassifier_28062023_20_01.pickle"

# Load the model
model, features = my_func.load_pickle(pickle_file)

In [ ]:
fea_lgb = model.feature_name_
fea_lgb

In [8]:
# Get feature importances
X = df_train_fe.drop(['target', 'abon_id'], axis=1)
importances = model.feature_importances_
feature_names = X.columns

# Create a DataFrame with feature names and importances
df_importances = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

# Sort the DataFrame by importance in descending order
df_importances = df_importances.sort_values(by='Importance', ascending=False)

# Print the DataFrame
print(df_importances)




                       Feature  Importance
632           loc_market_share         116
606                         lt          61
634          device_days_usage          45
604                Balance_uah          41
705                day_end_gba          32
..                         ...         ...
620  DNZ_COUNT_open_loan_year5           0
264   clc_no_vas_roam_min_mnt1           0
623                        Fax           0
263       all_roam_clc_mea_wk1           0
0    Ama_rchrgmnt_sum_max_mnt1           0

[815 rows x 2 columns]


In [12]:
df_imp=df_importances[df_importances['Importance']>0]

In [2]:
# Get the list of selected features
#selected_features = df_imp['Feature'].tolist()

# Calculate the Spearman correlation for the selected features
#correlation_matrix = df_train_fe[selected_features].corr(method='spearman')

# Display the correlation matrix
correlation_matrix

NameError: name 'correlation_matrix' is not defined

In [14]:
def calculate_spearman_correlation(df):
    """
    Calculate the Spearman correlation matrix of a DataFrame.

    Parameters:
    - df (DataFrame): The input DataFrame.

    Returns:
    - correlation_matrix (DataFrame): A DataFrame showing the Spearman correlation matrix.
    """
    correlation_matrix = df.corr(method='spearman')
    return correlation_matrix


In [30]:
# Create a mask for the upper triangle of the correlation matrix
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)

# Apply the mask to the correlation matrix
corr_matrix_masked = correlation_matrix.where(mask)

# Get the features with correlations greater than 0.9
to_drop = [column for column in corr_matrix_masked.columns if any(corr_matrix_masked[column].abs() > 0.9)]

# Get the selected features by removing the features to drop from the list of all features
selected_features = [feature for feature in selected_features if feature not in to_drop]

# Print the selected features
print('Selected features with absolute correlations less than 0.9:')
print(selected_features)


Selected features with absolute correlations less than 0.9:
[]


In [32]:
# Create a mask for the upper triangle of the correlation matrix
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)

# Apply the mask to the correlation matrix
corr_matrix_masked = correlation_matrix.where(mask)

# Get the pairs of features with correlations greater than 0.9
high_corr_pairs = [(column, row) for column in corr_matrix_masked.columns for row in corr_matrix_masked.index if abs(corr_matrix_masked[column][row]) > 0.9]

# Print the pairs of features with high correlation
print('Pairs of features with absolute correlations greater than 0.9:')
for pair in high_corr_pairs:
    print(pair)

# Get the features to drop by extracting the unique features from the pairs
to_drop = list(set([feature for pair in high_corr_pairs for feature in pair]))

# Get the selected features by removing the features to drop from the list of all features
selected_features = [feature for feature in selected_features if feature not in to_drop]

# Print the selected features
print('\nSelected features with absolute correlations less than 0.9:')
print(selected_features)


Pairs of features with absolute correlations greater than 0.9:
('imei_mean_days_usage', 'imei_mean_long_days_usage')
('days_of_end_last_ppm', 'days_of_last_ppm')
('device_width_mm', 'device_height_mm')
('num_act_days_std_mnt3', 'num_act_days_mea_mnt3')
('all_cnt_max_mnt3', 'all_cnt_std_mnt3')
('num_act_days_min_mnt3', 'num_act_days_mea_mnt3')
('num_act_days_min_mnt3', 'num_act_days_std_mnt3')
('all_cnt_mea_mnt3', 'all_cnt_max_mnt3')
('non_accum_internet_vol_mea_mnt1', 'non_accum_internet_vol_mea_wk1')
('pay_avg_td_mnt3', 'pay_max_td_mnt3')
('non_accum_internet_vol_std_mnt1', 'non_accum_internet_vol_mea_mnt1')
('voice_mts_in_drest_part_max_mnt3', 'voice_mts_in_drest_part_std_mnt3')
('voice_in_cmpttrs_avg_durmax_mnt3', 'voice_in_cmpttrs_avg_durmea_mnt3')
('voice_in_fix_tar_dur_mea_mnt3', 'voice_in_fix_tar_dur_max_mnt3')
('conn_com_part_mea_mnt3', 'com_num_part_mea_mnt3')
('data_3g_tv_cnt_max_mnt3', 'data_3g_tv_cnt_std_mnt3')
('sms_in_cnt_mea_mnt3', 'sms_in_cnt_std_mnt3')
('voice_in_td_cn

In [33]:
# Create a mask for the upper triangle of the correlation matrix
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool), k=1)

# Apply the mask to the correlation matrix
corr_matrix_masked = correlation_matrix.where(mask)

# Get the pairs of features with correlations greater than 0.9
high_corr_pairs = [(column, row) for column in corr_matrix_masked.columns for row in corr_matrix_masked.index if abs(corr_matrix_masked[column][row]) > 0.9]

# Print the pairs of features with high correlation
print('Pairs of features with absolute correlations greater than 0.9:')
for pair in high_corr_pairs:
    print(pair)

# Get the features to drop by extracting the first feature from each pair
to_drop = list(set([pair[0] for pair in high_corr_pairs]))

# Get the selected features by removing the features to drop from the list of all features
selected_features = [feature for feature in selected_features if feature not in to_drop]

# Print the selected features
print('\nSelected features with absolute correlations less than 0.9:')
print(selected_features)


Pairs of features with absolute correlations greater than 0.9:
('imei_mean_days_usage', 'imei_mean_long_days_usage')
('days_of_end_last_ppm', 'days_of_last_ppm')
('device_width_mm', 'device_height_mm')
('num_act_days_std_mnt3', 'num_act_days_mea_mnt3')
('all_cnt_max_mnt3', 'all_cnt_std_mnt3')
('num_act_days_min_mnt3', 'num_act_days_mea_mnt3')
('num_act_days_min_mnt3', 'num_act_days_std_mnt3')
('all_cnt_mea_mnt3', 'all_cnt_max_mnt3')
('non_accum_internet_vol_mea_mnt1', 'non_accum_internet_vol_mea_wk1')
('pay_avg_td_mnt3', 'pay_max_td_mnt3')
('non_accum_internet_vol_std_mnt1', 'non_accum_internet_vol_mea_mnt1')
('voice_mts_in_drest_part_max_mnt3', 'voice_mts_in_drest_part_std_mnt3')
('voice_in_cmpttrs_avg_durmax_mnt3', 'voice_in_cmpttrs_avg_durmea_mnt3')
('voice_in_fix_tar_dur_mea_mnt3', 'voice_in_fix_tar_dur_max_mnt3')
('conn_com_part_mea_mnt3', 'com_num_part_mea_mnt3')
('data_3g_tv_cnt_max_mnt3', 'data_3g_tv_cnt_std_mnt3')
('sms_in_cnt_mea_mnt3', 'sms_in_cnt_std_mnt3')
('voice_in_td_cn

In [37]:
selected_features

[]